In [1]:
import subprocess as sp
import sys,os,shutil,pickle,time
from os.path import join
from os.path import expanduser


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('samsungvaluemoney.csv')
df.head(5)

,stars,comment
0,5,S10 plus true all rounder with all boxes ticke...
1,5,\n\n\n\n\n\n\n\n\n\n \n \n \n Mesmerizin...
2,5,\n\n\n\n\n\n\n\n\n\n \n \n \n Very good\...
3,5,\n\n\n\n\n\n\n\n\n\n \n \n \n Nice\n\n \n
4,4,\n\n\n\n\n\n\n\n\n\n \n \n \n Value for ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1060 entries, 0 to 1059
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stars    1060 non-null   int64 
 1   comment  1060 non-null   object
dtypes: int64(1), object(1)
memory usage: 16.7+ KB


In [5]:
df.describe().round(1)

,stars
count,1060.0
mean,4.9
std,0.3
min,4.0
25%,5.0
50%,5.0
75%,5.0
max,5.0


In [6]:
# Determine how many missing values exist in the collection, in which case you can use .sum() chained onto is.na()
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0


In [7]:
df= df.dropna()
df.shape

(1060, 2)

In [8]:
#pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
df['stopwords'] = df['comment'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['comment','stopwords']].head()

,comment,stopwords
0,S10 plus true all rounder with all boxes ticke...,44
1,\n\n\n\n\n\n\n\n\n\n \n \n \n Mesmerizin...,1
2,\n\n\n\n\n\n\n\n\n\n \n \n \n Very good\...,0
3,\n\n\n\n\n\n\n\n\n\n \n \n \n Nice\n\n \n,0
4,\n\n\n\n\n\n\n\n\n\n \n \n \n Value for ...,1


In [10]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['comment'].apply(lambda x: count_punct(x))

In [11]:
df[['comment','punctuation']].head()

,comment,punctuation
0,S10 plus true all rounder with all boxes ticke...,16
1,\n\n\n\n\n\n\n\n\n\n \n \n \n Mesmerizin...,2
2,\n\n\n\n\n\n\n\n\n\n \n \n \n Very good\...,0
3,\n\n\n\n\n\n\n\n\n\n \n \n \n Nice\n\n \n,0
4,\n\n\n\n\n\n\n\n\n\n \n \n \n Value for ...,0


In [12]:
df['hastags'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df[['comment','hastags']].head()

,comment,hastags
0,S10 plus true all rounder with all boxes ticke...,0
1,\n\n\n\n\n\n\n\n\n\n \n \n \n Mesmerizin...,0
2,\n\n\n\n\n\n\n\n\n\n \n \n \n Very good\...,0
3,\n\n\n\n\n\n\n\n\n\n \n \n \n Nice\n\n \n,0
4,\n\n\n\n\n\n\n\n\n\n \n \n \n Value for ...,0


In [13]:
df.hastags.loc[df.hastags != 0].count()

0

In [14]:
df['numerics'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['comment','numerics']].head()

,comment,numerics
0,S10 plus true all rounder with all boxes ticke...,0
1,\n\n\n\n\n\n\n\n\n\n \n \n \n Mesmerizin...,0
2,\n\n\n\n\n\n\n\n\n\n \n \n \n Very good\...,0
3,\n\n\n\n\n\n\n\n\n\n \n \n \n Nice\n\n \n,0
4,\n\n\n\n\n\n\n\n\n\n \n \n \n Value for ...,0


In [15]:
df['upper'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['comment','upper']].head()

,comment,upper
0,S10 plus true all rounder with all boxes ticke...,4
1,\n\n\n\n\n\n\n\n\n\n \n \n \n Mesmerizin...,0
2,\n\n\n\n\n\n\n\n\n\n \n \n \n Very good\...,0
3,\n\n\n\n\n\n\n\n\n\n \n \n \n Nice\n\n \n,0
4,\n\n\n\n\n\n\n\n\n\n \n \n \n Value for ...,0


In [16]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['comment'].head()

0    s10 plus true all rounder with all boxes ticke...
1    mesmerizing display, excellent cameras, value ...
2                                            very good
3                                                 nice
4                                      value for money
Name: comment, dtype: object

In [17]:
df['comment'] = df['comment'].str.replace('[^\w\s]','')
df['comment'].head()

<ipython-input-17-66a30b7fedcd>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comment'] = df['comment'].str.replace('[^\w\s]','')


0    s10 plus true all rounder with all boxes ticke...
1    mesmerizing display excellent cameras value fo...
2                                            very good
3                                                 nice
4                                      value for money
Name: comment, dtype: object

In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['comment'].sample(10)

928                          purchased 39999 value money
512                                                 good
822                                                 good
409                                          value money
677                            awesome phone price range
885    recently got mobile phone far working excellen...
939                                          value money
169                                          value money
657                            awesome phone price range
170    s10 plus true rounder boxes ticked1screen10 pl...
Name: comment, dtype: object

In [19]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

In [20]:
# remove all urls from df
import re
import string

df['comment'] = df['comment'].apply(lambda x: remove_url(x))

In [21]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [22]:
# remove all html tags from df
df['comment'] = df['comment'].apply(lambda x: remove_html(x))

In [23]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [24]:
#Example
remove_emoji("Omg another Earthquake 😔😔")

'Omg another Earthquake '

In [25]:
# remove all emojis from df
df['comment'] = df['comment'].apply(lambda x: remove_emoji(x))

In [26]:
!pip install emot

Defaulting to user installation because normal site-packages is not writeable


In [27]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [28]:
#Example
remove_emoticons("Hello :-)")

'Hello '

In [29]:
df['comment'] = df['comment'].apply(lambda x: remove_emoticons(x))

In [30]:
from textblob import TextBlob
df['comment'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    s plus true rounder boxes ticked1screen10 plus...
1     mesmerizing display excellent camera value money
2                                                 good
3                                                 nice
4                                          value money
Name: comment, dtype: object

In [31]:
# We could do some of the cleaning steps as a sum of opreation like this:

# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [32]:
df['comment'] = df.comment.apply(round1)
df.comment

0        plus true rounder boxes      plusoverall sams...
1       mesmerizing display excellent cameras value money
2                                                    good
3                                                    nice
4                                             value money
                              ...                        
1055    recently got mobile phone far working excellen...
1056                           fantastic phonevalue money
1057                            awesome phone price range
1058                               purchased  value money
1059                                          value money
Name: comment, Length: 1060, dtype: object

In [33]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [34]:
df['comment'] = df.comment.apply(round2)
df.comment

0        plus true rounder boxes      plusoverall sams...
1       mesmerizing display excellent cameras value money
2                                                    good
3                                                    nice
4                                             value money
                              ...                        
1055    recently got mobile phone far working excellen...
1056                           fantastic phonevalue money
1057                            awesome phone price range
1058                               purchased  value money
1059                                          value money
Name: comment, Length: 1060, dtype: object

In [35]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:20]
freq

phone        848
money        636
value        530
great        318
first        212
plus         212
samsung      212
working      106
well         106
awesome      106
less         106
x            106
mobile       106
fantastic    106
hands        106
rounder      106
screen       106
chose        106
range        106
cameras      106
dtype: int64

In [36]:
df['word_count'] = df['comment'].apply(lambda x: len(str(x).split(" ")))
df[['comment','word_count']].head()

,comment,word_count
0,plus true rounder boxes plusoverall sams...,57
1,mesmerizing display excellent cameras value money,6
2,good,1
3,nice,1
4,value money,2


In [37]:
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0
stopwords,0,0.0
punctuation,0,0.0
hastags,0,0.0
numerics,0,0.0
upper,0,0.0
word_count,0,0.0


In [38]:
df['char_count'] = df['comment'].str.len() ## this also includes spaces
df[['comment','char_count']].head()

,comment,char_count
0,plus true rounder boxes plusoverall sams...,319
1,mesmerizing display excellent cameras value money,49
2,good,4
3,nice,4
4,value money,11


In [39]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))

In [40]:
df['avg_word'] = df['comment'].apply(lambda x: avg_word(x)).round(1)
df[['comment','avg_word']].head()

,comment,avg_word
0,plus true rounder boxes plusoverall sams...,5.4
1,mesmerizing display excellent cameras value money,7.3
2,good,4.0
3,nice,4.0
4,value money,5.0


In [41]:
df.sample(2)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
85,5,recently got mobile phone far working excellen...,5,1,0,0,1,8,54,5.9
74,4,value money,1,0,0,0,0,2,11,5.0


In [42]:
list(df)

['stars',
 'comment',
 'stopwords',
 'punctuation',
 'hastags',
 'numerics',
 'upper',
 'word_count',
 'char_count',
 'avg_word']

In [43]:
df.sample(5)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
588,5,purchased value money,1,3,0,0,0,4,22,6.3
548,5,purchased value money,1,3,0,0,0,4,22,6.3
989,5,value money,1,0,0,0,0,2,11,5.0
602,5,good,0,0,0,0,0,1,4,4.0
99,5,value money,1,0,0,0,0,2,11,5.0


In [44]:
add_words = ["get",     
"use",     
"first",      
"like","u","goes","im"      
]

stop_words = set(stopwords.words("english"))
stop_added = stop_words.union(add_words)

In [45]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_added))
df['comment'].sample(10)

155     recently got mobile phone far working excellen...
27                              awesome phone price range
749                                 purchased value money
454                                           value money
590     plus true rounder boxes plusoverall samsung ga...
37      recently got mobile phone far working excellen...
620     plus true rounder boxes plusoverall samsung ga...
549                                           value money
292                                                  good
1019                                          value money
Name: comment, dtype: object

In [46]:
df1= df

In [47]:
mask = df1.comment.str.endswith('br') 
df1.loc[mask, 'comment'] = df1.loc[mask, 'comment'].str[:-2]

In [48]:
df1['comment'] = df1['comment'].str.rstrip('tty')

In [49]:
df1['comment'].apply(lambda x: x[:-2] if x.endswith('tty') else x)

0       plus true rounder boxes plusoverall samsung ga...
1        mesmerizing display excellent cameras value mone
2                                                    good
3                                                    nice
4                                              value mone
                              ...                        
1055    recently got mobile phone far working excellen...
1056                            fantastic phonevalue mone
1057                            awesome phone price range
1058                                 purchased value mone
1059                                           value mone
Name: comment, Length: 1060, dtype: object

In [50]:
df1.loc[df1.comment.str.endswith('br'), 'comment']


Series([], Name: comment, dtype: object)

In [51]:
df1.loc[df1.punctuation >= 1000].comment.tolist()

[]

In [52]:
df.loc[df.punctuation >= 1000].comment.tolist()


[]

In [53]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:50]
freq

phone           848
value           530
mone            530
great           318
plus            212
samsung         212
working         106
mesmerizing     106
well            106
awesome         106
less            106
x               106
mobile          106
fantastic       106
hands           106
rounder         106
screen          106
punchy          106
range           106
cameras         106
phonevalue      106
bright          106
confidence      106
iphone          106
display         106
india           106
plusoverall     106
finally         106
thing           106
nice            106
chose           106
thought         106
full            106
thisactually    106
recently        106
purchased       106
price           106
colors          106
say             106
excellently     106
boxes           106
way             106
near            106
excellent       106
second          106
would           106
far             106
good            106
true            106
happy           106


In [54]:
df.to_csv('samsungvaluemoney2.csv', index=False)